<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# Naas - Send notification

**Tags:** #naas #emailbuilder #growth-engine

**Author:** [Florent Ravenel](https://www.linkedin.com/in/ACoAABCNSioBW3YZHc2lBHVG0E_TXYWitQkmwog/)

**Description:** This notebook send a notification with your growth performance.

## Input

### Import libraries

In [ ]:
from naas_drivers import emailbuilder, gsheet
import naas
import os
import naas_data_product
from datetime import date, datetime, timedelta
from naas_drivers import naasauth
import pandas as pd

### Setup variables
**Inputs**
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `input_dir`: Input directory to retrieve file from.
- `datalake_dir`: Datalake directory to retrieve plugin json file.

**Outputs**
- `email_to`: This variable is used for storing a list of email addresses that will receive the notification email. In this case, the list contains the email address of the script user, which is retrieved using the naasauth.connect().user.me().get("username") method.
- `subject`: This variable is used for storing the subject of the notification email. In this case, the subject is "🖼️ Content Engine: LinkedIn profile update".

In [ ]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "growth-engine", date.today().isoformat())
input_image = "growth_trend.png"
input_html = "growth_trend.html"
datalake_dir = naas_data_product.OUTPUTS_PATH
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_name_input = "GROWTH"

# Outputs
email_to = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entity"), "emails") or []
output_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "content-engine", date.today().isoformat())

## Model

### Set outputs

In [ ]:
entity_name = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entity"), "entity_name") or ""
timestamp_email = datetime.now(TIMEZONE).strftime('%Y-%m-%d %H:%M:%S')
timestamp_csv = datetime.now(TIMEZONE).strftime('%Y%m%d%H%M%S')
email_subject = f"🚀 Growth Assistant Report - {entity_name} ({timestamp_email})"
csv_output = os.path.join(output_dir, f"{timestamp_csv}_{TW}_growth-assistant-report_{entity_name.lower().replace(' ', '_')}.csv")
print("- CSV output:", csv_output)

### Get assets

In [ ]:
image_url = get_image_asset(input_dir, input_image)
print("- Image URL:", image_url)

image_html = get_image_asset(input_dir, input_html)
print("- Image HTML:", image_html)

plugin_url = naas.asset.find(os.path.join(datalake_dir, "plugins", "🚀_growth_assistant.json"))
print("- Plugin URL:", plugin_url)

### Get data

In [ ]:
df_input = pload(input_dir, "growth")    
if df_input is None:
    df_input = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name_input)

# Filter on this week
df_input = df_input[df_input["SCENARIO"].isin([TW])]

# Display result
df_input.to_csv(csv_output, index=False)
print(f"- Leads generated {TW}:", len(df_input))
# df_input.head(1)

### Create 'Performance' details

In [ ]:
# Init
kpis = []

# Get detailed data
data = pload(input_dir, "growth_trend")

# Get aggregated data
df = data.groupby(["LABEL", "LABEL_D"], as_index=False).agg({"VALUE": "sum"})
df = df.sort_values(by="LABEL", ascending=False).reset_index(drop=True)

# Get main kpis
for row in df.itertuples():
    index = row.Index
    label = row.LABEL
    label_d = row.LABEL_D
    value = row.VALUE
    value_n1 = 0
    
    # Get value n1
    if index + 1 < len(df):
        value_n1 = df.loc[index + 1, "VALUE"]
        
    # Calc variation in value and create string to be displayed
    varv = value - value_n1
    if varv > 0:
        varv_d = f" (+{format_number(varv)})"
    elif varv == 0:
        varv_d = ""
    else:
        varv_d = f" (-{format_number(varv)})"
    main_kpi = f"{label_d} : <b>{format_number(value)}{varv_d}</b><span style='font-size: 15px;'>"
    
    # Get details kpis
    tmp_df = data[data["LABEL"] == label].sort_values(by="ORDER", ascending=False).reset_index(drop=True)
    for row_d in tmp_df.itertuples():    
        group = row_d.GROUP
        value_group = row_d.VALUE
        if value == 0:
            pourcentage = 0
        else:
            pourcentage = round(value_group * 100 / value)
        main_kpi = f"{main_kpi}<ul>{group}: {format_number(value_group)} ({format_number(pourcentage)}%)</ul>"
    main_kpi = f"{main_kpi}</span>"
    kpis.append(main_kpi)
print(kpis)

### Create Email brief

In [ ]:
def email_brief(
    kpis,
    plugin_url,
    image_url,
    image_html,
):
    content = {
        'heading': "🚀 Growth Assistant",
        "txt_intro": (
            f"Hi {entity_name.split(' ')[0]},<br><br>" "I hope this email finds you well. Here is the latest update on your growth performance."
        ),
        "title_1": emailbuilder.text(
            "Overview", font_size="25px", text_align="center", bold=True
        ),
        "image_1": emailbuilder.image(image_url, image_html),
        "title_2": emailbuilder.text(
            "Performance", font_size="25px", text_align="center", bold=True
        ),
        "text_1": emailbuilder.text(
            "Statistics vs last week:",
            underline=True,
        ),
        "list_1": emailbuilder.list(kpis),
        "button_1": emailbuilder.button(
            link=(f"https://naas.ai/chat/use?plugin_url={plugin_url}"),
            text="Start Chatting",
            background_color="#181a1c",
        ),
        "footer_cs": emailbuilder.footer_company(naas=True),
    }

    email_content = emailbuilder.generate(display="iframe", **content)
    return email_content


email_content = email_brief(
    kpis,
    plugin_url,
    image_url,
    image_html
)

## Output

### Send the email

In [ ]:
if len(email_to) > 0:
    naas.notification.send(
        email_to=email_to,
        subject=email_subject,
        html=email_content,
        files=[csv_output],
    )